## Distributions and the Central Limit Theorem

Today we are going to introduce:
- Statistical distributions
- The Central Limit Theorem

In [1]:
import pandas as pd
import numpy as np
import altair as alt
import helpers.plotting as pt
from helpers.svg_wrapper import SVGImg
pt.enable_slide_theme()
pt.import_lato_font_in_notebook()

In [2]:
%%html
<script type="text/x-mathjax-config">
MathJax.Hub.Config({
    "HTML-CSS" : {
        mtextFontInherit: true,
    }
});
</script>

In [3]:
def distribution_parameter(
        min:float, max:float, step:float, init_value:float, name:str, type:float='range'
    ) -> alt.selection:
    b = alt.binding(
        input=type, min=min, max=max, step=step, name=name
    )
    return alt.selection_single(
        bind = b,
        init = dict(value=init_value)
    )

def distribution_chart(
        xmin:float,
        xmax:float,
        xstep:float,
        pdf_transform_expression: str,
        mark_type='line',
        xscale=alt.Scale(),
        yscale=alt.Scale(),
        **distribution_parameters
    ) -> alt.Chart:
    return alt.Chart(
        alt.sequence(xmin, xmax, xstep, as_='x')
    ).__getattr__(f'mark_{mark_type}')().encode(
        x=alt.X('x:Q', scale=xscale),
        y=alt.Y('px:Q', scale=yscale),
    ).add_selection(
        *list(distribution_parameters.values())[::-1], 
    ).transform_calculate(
        **{k: v.value for k, v in distribution_parameters.items()}
    ).transform_calculate(
        px = pdf_transform_expression
    )

def chart_with_parameters(chart, **distribution_parameters):
    return chart.add_selection(
        *list(distribution_parameters.values()), 
    ).transform_calculate(
        **{k: v.value for k, v in distribution_parameters.items()}
    )

### How to describe the possible outcomes of an experiment? Distributions!

Think about **throwing a** six-sided **[die](https://en.wikipedia.org/wiki/Dice)**: 
- The possible outcomes are the integers (whole numbers) 1, 2, 3, 4, 5, 6.
- Each outcome should have the same **probability: 1/6**.
- The outcomes are described by the **discrete uniform distribution** on the integers 1-6.

In [4]:
SVGImg('images/die.svg', width='15%', output_dir='slides')

## There are discrete and continuous probability distributions!

For simplicity, we will keep the uniform distribution as an example for the moment.

### In the discrete case, 
- we can specify the probability of each possible outcome.
- Mathematically this is done via a "[probability mass function](https://en.wikipedia.org/wiki/Probability_mass_function)".
- All the probabilities add up to one (one out of all possible outcomes will happen).

In [5]:
# uniform probability mass function
distribution_chart(
    pdf_transform_expression = '1/datum.max * (1 <= datum.x) * (datum.x <= datum.max)',
    xmin = -1,
    xmax = 21,
    xstep = 1,
    mark_type='bar',
    xscale=alt.Scale(domain=(0,21)),
    yscale=alt.Scale(domain=(0,0.51)),
    max=distribution_parameter(
        min=2,
        max=20,
        step=1,
        init_value=2,
        name='Max'
    )
)

alt.Chart(...)

Above: the discrete uniform distribution with minimum 2, maximum set by slider.

### A continuous example: a spinning disc with markings that can rotate freely.

- If you spin it, it will stop at a random angle.
- If we say it stopped at e.g. 18°, we mean it was closest to that marking.
- There are infinitely many angles between 17.5° and 18.5°.
- The probability to hit a specific one is zero.
- The probability to end up somewhere in this 1° intervall, is 1/360!

In [6]:
SVGImg('images/spinner.svg', width='50%', output_dir='slides')

### For a continuous distribution, 

- There are infinitely many possible outcomes.
- The [probability density](https://en.wikipedia.org/wiki/Probability_density_function) describes which outcomes may occur.
- The **probability** for an outcome to be in an interval <br>is equal to the **area** under the probability density.

### Example: the continuous uniform distribution

In [7]:
# uniform density
xmax = 3 # upper limit
dx = xmax / 1000 # step size for plotting

# value ranges to show on the axes
xscale = alt.Scale(domain=(-0.1, 3.1), nice=False)
yscale = alt.Scale(domain=(0,2))
pscale = alt.Scale(domain=(0,1.1))

# interactive parameters
sel_min = distribution_parameter(
    min=0,
    max=xmax,
    step=0.1,
    init_value=0,
    name='Selection min'
)
sel_max = distribution_parameter(
    min=0,
    max=xmax,
    step=0.1,
    init_value=xmax+0,
    name='Selection max'
)
dist_max = distribution_parameter(
    min=0.5,
    max=xmax,
    step=0.1,
    init_value=2,
    name='Distribution max'
)

# all parts of the chart
base = alt.Chart(
    alt.sequence(start=-0.1, stop=xmax+0.1, step=dx, as_='x')
)
density = base.mark_line().encode(
    x=alt.X('x:Q', scale=xscale, title='x'),
    y=alt.Y('px:Q', scale=yscale, title='Probability Density'),
).properties(
    width=400
)
selected = base.mark_area(color='LightBlue', opacity=0.5).encode(
    x=alt.X('x:Q'),
    y=alt.Y('ps:Q', title='Probability Density'),
)
selection_min = base.mark_rule(color='SlateGrey').encode(
    x=alt.X('mean(sel_min):Q', title=''),
    opacity=alt.condition(sel_min.value > 0, alt.value(1), alt.value(0))
)
selection_max = base.mark_rule(color='SlateGrey').encode(
    x=alt.X('mean(sel_max):Q', title=''),
    opacity=alt.condition(sel_max.value < xmax, alt.value(1), alt.value(0))
)
selected_area = base.encode(
    y=alt.Y('sum(probability):Q', scale=pscale, title='Probability in Selection'),
).mark_bar(clip=True, color='LightBlue').properties(
        width=50
)
# combine
chart_with_parameters(
    alt.hconcat(selection_min + selection_max + selected + density  | selected_area),
    sel_max=sel_max,
    sel_min=sel_min,
    dist_max=dist_max,
).transform_calculate(
    px = f'1 / datum.dist_max * ((datum.x > 0 ) & (datum.x < datum.dist_max))',
    ps = f'datum.px * ((datum.x > datum.sel_min ) & (datum.x < datum.sel_max))',
    probability = f'datum.ps*{dx}'
).display(renderer='svg')

alt.HConcatChart(...)

Left: the continuous uniform distribution (line) between 0 and a variable maximum.<br>
Right: the probability that an outcome falls between selection min and selection max.

## There are many different probability distributions!

Let's look at a very common one: the [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution).

## The normal distribution...

- is also called the bell curve or Gaussian distribution. 
- It is specified by its mean and its variance or standard deviation.

In [8]:
# gaussian density
distribution_chart(
    pdf_transform_expression = 'densityNormal(datum.x, datum.mean, datum.std)',
    xmin = -5,
    xmax = 5,
    xstep = .1,
    mean = distribution_parameter(
        min=-5,
        max=5,
        step=.1,
        init_value=0,
        name='Mean'
    ),
    std = distribution_parameter(
        min=.5,
        max=5,
        step=.1,
        init_value=1,
        name='Std'
    )
).properties(
        width=500
)

alt.Chart(...)

### When we sample from a normal distribution...

- the standard deviation tells us which fraction of the observations we expect to fall how far away from the mean.
- This is called the [68-95-99 rule](https://en.wikipedia.org/wiki/68–95–99.7_rule).
- [Remember](2_practical_basics.slides.html) that the [standard error](https://en.wikipedia.org/wiki/Standard_error) is modeled as a normal distribution, so the same rule applies there.

In [9]:
SVGImg('images/68-95-99.svg', width='50%', output_dir='slides')

### The sums of independent random numbers are often approximately normally distributed!

- This is why the normal distribution is so common.
- Random fluctuations and errors in particular are often caused<br> by many independent contributions.

### Example: the sum of the points from several dice

In [14]:
import itertools

In [15]:
# a naive brute-force calculation of the probability mass function for the sum of n dice
# ======================================================================================
n_max = 7 # maximum number of dice to consider
res = pd.DataFrame(index=pd.Index(range(1,6*n_max+1), name='Sum of Points'))
# for n = 1 ... n_max dice
for n in range(1, n_max+1):
    # calculate number of possibilities to get each possible outcome value
    outcome, possibilities = np.unique(
        # sum of points for each outcome
        np.array(
            # create list of all possible outcomes
            # each outcome is a list of the number of points for each die
            list(itertools.product(*[list(range(1,7))]*n))
        ).sum(1), 
        return_counts=True
    )
    res.loc[outcome, n] = possibilities

# normalise counts to probabilities
res /= res.sum()

# add mean points per die
dice_sum_df = pd.melt(res.reset_index(), id_vars=['Sum of Points'], var_name='Dice', value_name='Probability')
dice_sum_df['Points per Die'] = dice_sum_df['Sum of Points'] / dice_sum_df['Dice']

In [16]:
# plot parameters
n_dice = distribution_parameter(
    min=1,
    max=n_max,
    step=1,
    init_value=1,
    name='Dice'
)
measure_dropdown = alt.binding_select(
    options=['Sum of Points', 'Points per Die'], 
    name='Measure:'
)
selection = alt.selection_single(
    bind=measure_dropdown, 
    fields=['Measure'], 
    init=dict(Measure = 'Sum of Points')
)

# chart base
chart = alt.Chart(
    dice_sum_df
).mark_bar().encode(
    x=alt.X('Value:Q', title='Points'),
    y='Probability:Q'
)
# add parameters & filters
chart_with_parameters(
    chart,
    n_dice=n_dice,
).transform_filter(
    +alt.datum.Dice == +n_dice.value,
).transform_fold(
    # transform the two measure columns to long format
    # i.e. measure denotes from which column the value came
    ['Sum of Points', 'Points per Die'],
    as_=['Measure', 'Value']
).add_selection(
    selection
).transform_filter(
    selection
).properties(
    width=250
).display(renderer='svg')

alt.Chart(...)

Above: as the number of dice is increased, the normal distribution emerges.

### The Central Limit Theorem

Take a random sample of
- N independent observations 
- with a finite variance.

In the limit of large N, the distribution of

$$\frac{\text{Sample Mean} - \text{Population Mean}}{\sqrt{\text{Population Variance } / \text{ N}}}$$

converges towards a standard normal distribution (i.e. mean 0 and variance 1)

See also: [Mathworld](https://mathworld.wolfram.com/CentralLimitTheorem.html), [Wikipedia](https://en.wikipedia.org/wiki/Central_limit_theorem)

## The Central Limit Theorem has many applications, but...

- Some distributions need more samples for a good normal approximation.
- Correlations slow down the convergence.
- Not all distributions have a finite variance.
- Not all distributions have a finite mean.

#### Therefore...
- make sure to always look at your raw numbers. 
- Consider a [normality test](https://en.wikipedia.org/wiki/Normality_test) or ask a statistician.
- We will have another session on what to do if your data is not normal.

## Examples where sums don't converge towards a normal distribution
- The [Cauchy distribution](https://en.wikipedia.org/wiki/Cauchy_distribution) and [Lévy distribution](https://en.wikipedia.org/wiki/Lévy_distribution) remain stable under aggregation. I.e. they don't converge towards a normal distribution.
- Price changes in financial markets have a (just) finite variance but it [changes](https://en.wikipedia.org/wiki/Heteroscedasticity) over time. E.g. Daily price changes are sums of many small ones, but even their logarithms exhibit much more [extremes](https://www.statisticshowto.com/heavy-tailed-distribution/) than a expected normal distribution.

#### See also: 
- [This game](http://seesaw.neuro.uni-bremen.de) generates extreme events.
- An Explanation of [infinite mean and variance](https://stats.stackexchange.com/a/91515)

## Summary

- There are discrete 🎲 and continuous ⚪️ random distributions
- For continuous distributions, the area under the probability density gives us a probability 🗻
- Sums of many independent random variables with finite variance converge towards a normal distribution 🔔
- Correlations or extreme events can lead to exceptions 🤪

To wrap up, check ot the [Galton board](https://en.wikipedia.org/wiki/Bean_machine), e.g in this [video](https://www.youtube.com/watch?v=Vo9Esp1yaC8)

## In the next session,

we will take a deeper dive into…

- comparing two different statistical distributions,
- statistical significance and p-values,
- statistical power and effect sizes.